# Text Classification for the IMDB Dataset using DL
**Objective:** classify the IMDB Reviews into positive or negative. <br>
In this notebook we explore different DL-based text classification models and compare their performance. <br>
The notebook is coded with Keras and explores the following three architectures:
1. CNN-based models with and without pre-trained embeddings
2. LSTM-based models with and without pre-trained embeddings
3. Transformer-based models with and without pre-trained embeddings (for you to do)
This notebook needs a GPU; google colab could be used.
**Useful documentation** <br>
- [Pre-trained embeddings with Keras](https://keras.io/examples/nlp/pretrained_word_embeddings/) 
- [Sentiment classification with LSTM keras](https://slundberg.github.io/shap/notebooks/deep_explainer/Keras%20LSTM%20for%20IMDB%20Sentiment%20Classification.html) 
# Installation of needed libraries

In [1]:
# !pip install numpy==1.19.5
# !pip install tensorflow

# Importing needed libraries

In [2]:
import os, sys, numpy as np, pandas as pd
from zipfile import ZipFile
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Input, GlobalMaxPooling1D
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Embedding, LSTM
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.initializers import Constant

# Downloading dataset & pre-trained GLOVE embeddings
1. [GLOVE](http://nlp.stanford.edu/data/glove.6B.zip)
2. [IMDB dataset](http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz)

They are both zipped, thus we need to unzip them. <br>
We will put the data and pre-trained mebddings into a folder called Data.


In [3]:
if not os.path.exists('Data'):
    os.mkdir('Data')
if not os.path.exists('Data/glove.6B') : 
    temp='glove.6B.zip' 
    file = ZipFile(temp)  
    file.extractall('Data/glove.6B') 
    file.close()

In [4]:
BASE_DIR = 'Data'
GLOVE_DIR = os.path.join(BASE_DIR, 'glove.6B')

df_train = pd.read_excel('train_set_imdb_reviews.xlsx')
df_test = pd.read_excel('test_set_imdb_reviews.xlsx')

# EDA
- Explore both datasets
- Clean the datasets: !!! The cleaning steps should be deduced following the exploration done on the train set not on the test set to garantee **no data leakage**. However, it is applied on both. 
- Check if the dataset is balanced or not 
- Bonus: fix the imbalance if it turns out to be the case

At the end of the EDA, set the cleaned reviews (texts) to the variables ``train_texts`` and ``test_texts`` and the sentiments to ``train_labels`` and ``test_labels``. <br>
If you failed this step, use the following commands: <br>
1. ``train_texts = df_train.reviews.apply(lambda x: str(x)).tolist()``
2. ``test_texts = df_train.reviews.apply(lambda x: str(x)).tolist()``
3. ``train_labels = df_train.sentiment.tolist()``
4. ``test_labels = df_test.sentiment.tolist()``

In [5]:
# ...
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import nltk
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[\W_]+', ' ', text)
    text = re.sub(r'\d+', '', text)
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    text = ' '.join(words)
    return text

[nltk_data] Downloading package stopwords to /Users/fulin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/fulin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
## uncomment the following if you fail the cleaning
train_texts = df_train.reviews.apply(lambda x: str(x)).tolist()
test_texts = df_test.reviews.apply(lambda x: str(x)).tolist()

In [7]:
train_labels = df_train.sentiment.tolist()
test_labels = df_test.sentiment.tolist()
labels_index = {'pos':1, 'neg':0} 

# Tokenization of sentences using keras Tokenizer

In keras, unlike pytorch, the Tokenizer not only splits the sentence into words but also convert words into their ids.<br>
AS we have mentioned in class, keras is a high level layer on top of tensoflow implemented to allow novice DL users (more precisely traditional ML users) to develop DL models. <br>

**Remember**, the pre-processing is learnt by looking at the train dataset only to garantee **no data leakage**, and it is applied on both datasets. 
&rarr; we fit the tokenizer on training data, then use it to tokenize both datasets. <br>


In [8]:
#Vectorize these text samples into a 2D integer tensor using Keras Tokenizer 
# 
MAX_NUM_WORDS = 20000 
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS) 
tokenizer.fit_on_texts(train_texts) 
train_sequences = tokenizer.texts_to_sequences(train_texts) #Converting text to a vector of word indexes 
test_sequences = tokenizer.texts_to_sequences(test_texts) 
word_index = tokenizer.word_index 
print('Found %s unique tokens.' % len(word_index))

Found 88580 unique tokens.


In [9]:
train_sequences[0]

[62,
 4,
 3,
 129,
 34,
 44,
 7575,
 1414,
 15,
 3,
 4252,
 514,
 43,
 16,
 3,
 633,
 133,
 12,
 6,
 3,
 1301,
 459,
 4,
 1751,
 209,
 3,
 10784,
 7693,
 308,
 6,
 676,
 80,
 32,
 2137,
 1110,
 3008,
 31,
 1,
 929,
 4,
 42,
 5120,
 469,
 9,
 2665,
 1751,
 1,
 223,
 55,
 16,
 54,
 828,
 1318,
 847,
 228,
 9,
 40,
 96,
 122,
 1484,
 57,
 145,
 36,
 1,
 996,
 141,
 27,
 676,
 122,
 1,
 13885,
 411,
 59,
 94,
 2278,
 303,
 772,
 5,
 3,
 837,
 11036,
 20,
 3,
 1755,
 646,
 42,
 125,
 71,
 22,
 235,
 101,
 16,
 46,
 49,
 624,
 31,
 702,
 84,
 702,
 379,
 3493,
 12996,
 2,
 16815,
 8422,
 67,
 27,
 107,
 3348]

Since we are dealing with a classical ML/DL model, input dimension should always be fixed. <br>
As in a traditional ML model, the number of attributes/features/columns should be fixed, in a DL model, the input dimension should be fixed as well. <br>
In our case, the input features are sentences i.e. list of words. In order to make sure that the input has a fixed size, i.e. the sentences having the same size, we will need to fix a max length (MAX_LEN) parameter, which is the maximum number of words composing a sentence. <br>
You might ask yourselves, But every sentence has a different set of words, shouldn't we create an input size that is equal to the number of unique words in our corpus? <br>
The answer is No, because, we will never deal with words, we will deal with embeddings such that all words are embedded with vectors having the same dimension $d$ &rarr; every sentence of our corpus will be transformed into an input of size MAX_LEN $\times d$ &rarr; our input will have the same size. <br>
Thus: <br>
- sentences with number of words > than MAX_LEN will be truncated; we chose a post truncating i.e., the first MAX_LEN are retained and the remaining words are removed. 
- sentences with number of words < than MAX_LEN will be padded; we chose a post padding i.e., the 0 id will be added after the ids of the words present in the sentence.

In [10]:
MAX_LEN = 1000
trainvalid_data = pad_sequences(sequences=train_sequences, maxlen=MAX_LEN, padding='post', truncating='post', value=0.0)
test_data = pad_sequences(sequences=test_sequences, maxlen=MAX_LEN, padding='post', truncating='post', value=0.0)

In [11]:
trainvalid_data[0], test_data[0]

(array([   62,     4,     3,   129,    34,    44,  7575,  1414,    15,
            3,  4252,   514,    43,    16,     3,   633,   133,    12,
            6,     3,  1301,   459,     4,  1751,   209,     3, 10784,
         7693,   308,     6,   676,    80,    32,  2137,  1110,  3008,
           31,     1,   929,     4,    42,  5120,   469,     9,  2665,
         1751,     1,   223,    55,    16,    54,   828,  1318,   847,
          228,     9,    40,    96,   122,  1484,    57,   145,    36,
            1,   996,   141,    27,   676,   122,     1, 13885,   411,
           59,    94,  2278,   303,   772,     5,     3,   837, 11036,
           20,     3,  1755,   646,    42,   125,    71,    22,   235,
          101,    16,    46,    49,   624,    31,   702,    84,   702,
          379,  3493, 12996,     2, 16815,  8422,    67,    27,   107,
         3348,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
      

# Converting the target into a categorical tensor variable for DL model training
Keras implements the command ``to_categorical``, it transforms each label into a one-hot encode array of dimension = unique number of categories and sets the value 1 on the index i if the data sample belongs to the category i else 0. With to categorical, if an input belongs to several categories at a time, the label would contain several 1. <br>
Here there is 2 catgories: neg and pos &rarr; the dimension is 2. <br>
Example: the target of a review with a pos review is converted with ``to_categorical`` to an ``array([0,1])``, while the target of a review with a neg review is converted with ``to_categorical`` to an ``array([1,0])``.

In [12]:
trainvalid_labels = to_categorical(np.asarray(train_labels))
test_labels = to_categorical(np.asarray(test_labels))

In [13]:
train_labels[12500] , trainvalid_labels[12500]

(1, array([0., 1.], dtype=float32))

# Split the training data into a training set and a validation set

In [14]:
VALIDATION_SPLIT = 0.2
indices = np.arange(trainvalid_data.shape[0])
np.random.shuffle(indices)
trainvalid_data = trainvalid_data[indices]
trainvalid_labels = trainvalid_labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * trainvalid_data.shape[0])
x_train = trainvalid_data[:-num_validation_samples]
y_train = trainvalid_labels[:-num_validation_samples]
x_val = trainvalid_data[-num_validation_samples:]
y_val = trainvalid_labels[-num_validation_samples:]

# Convert the token ids into embedding vectors
1. Extract embeddings from glove.6B.100d.txt
2. Convert the words in the dataset into embeddings using the dictionary from step 1
3. Create the embedding layer for keras; this will be the first layer of our DL model.

In [15]:
EMBEDDING_DIM = 100 
print('Preparing embedding matrix.')

# first, build index mapping words in the embeddings set to their embedding vector
#  every line in glove.6B.100d.txt contains the word followed by the embedding vector
embeddings_index = {}
with open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'),encoding='utf8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

print('Found %s word vectors in Glove embeddings.' % len(embeddings_index))

# prepare embedding matrix - rows are the words from word_index, columns are the embeddings of that word from glove.
num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i > MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

Preparing embedding matrix.
Found 400000 word vectors in Glove embeddings.


In [16]:
# load these pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed during training
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=MAX_LEN,
                            trainable=False)
print("Preparing of embedding matrix is done")

Preparing of embedding matrix is done


# Training and evaluating the DL model
We will test 3 DL models:
- 1D CNN-based architecture
- LSTM-based architecture
- Transformer-based architecture (to do it on your own)

### 1D CNN Model with pre-trained embedding

In [18]:
print('Define a 1D CNN model.')

cnnmodel = Sequential()
cnnmodel.add(embedding_layer)
cnnmodel.add(Conv1D(128, 5, activation='relu'))
cnnmodel.add(MaxPooling1D(5))
cnnmodel.add(Conv1D(128, 5, activation='relu'))
cnnmodel.add(MaxPooling1D(5))
cnnmodel.add(Conv1D(128, 5, activation='relu'))
cnnmodel.add(GlobalMaxPooling1D())
cnnmodel.add(Dense(128, activation='relu'))
cnnmodel.add(Dense(len(labels_index), activation='softmax'))

cnnmodel.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])
#Train the model. Tune to validation set. 
cnnmodel.fit(x_train, y_train,
          batch_size=128,
          epochs=1, validation_data=(x_val, y_val))
#Evaluate on test set:
score, acc = cnnmodel.evaluate(test_data, test_labels)
print('Test accuracy with CNN:', acc)

Define a 1D CNN model.
782/782 [==============================] - 14s 18ms/step - loss: 0.5203 - acc: 0.7785
Test accuracy with CNN: 0.7784799933433533


### 1D CNN model with training your own embedding
The only difference here is that the embedding layer we created ``embedding_layer`` using the pre-trained glove embeddings is no longer used here. We initialize an ambedding layer with randomly initialized weights ``Embedding(MAX_NUM_WORDS, 128)``.

In [19]:
print("Defining and training a CNN model, training embedding layer on the fly instead of using pre-trained embeddings")
cnnmodel = Sequential()
cnnmodel.add(Embedding(MAX_NUM_WORDS, 128))
cnnmodel.add(Conv1D(128, 5, activation='relu'))
cnnmodel.add(MaxPooling1D(5))
cnnmodel.add(Conv1D(128, 5, activation='relu'))
cnnmodel.add(MaxPooling1D(5))
cnnmodel.add(Conv1D(128, 5, activation='relu'))
cnnmodel.add(GlobalMaxPooling1D())
cnnmodel.add(Dense(128, activation='relu'))
cnnmodel.add(Dense(len(labels_index), activation='softmax'))

cnnmodel.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])
#Train the model. Tune to validation set. 
cnnmodel.fit(x_train, y_train,
          batch_size=128,
          epochs=1, validation_data=(x_val, y_val))
#Evaluate on test set:
score, acc = cnnmodel.evaluate(test_data, test_labels)
print('Test accuracy with CNN:', acc)

Defining and training a CNN model, training embedding layer on the fly instead of using pre-trained embeddings
782/782 [==============================] - 18s 23ms/step - loss: 0.3030 - acc: 0.8726
Test accuracy with CNN: 0.8726000189781189


### LSTM Model with training your own embedding 

In [20]:
print("Defining and training an LSTM model, training embedding layer on the fly")

#model
rnnmodel = Sequential()
rnnmodel.add(Embedding(MAX_NUM_WORDS, 128))
rnnmodel.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
rnnmodel.add(Dense(2, activation='sigmoid'))
rnnmodel.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
print('Training the RNN')

rnnmodel.fit(x_train, y_train,
          batch_size=32,
          epochs=1,
          validation_data=(x_val, y_val))
score, acc = rnnmodel.evaluate(test_data, test_labels,
                            batch_size=32)
print('Test accuracy with RNN:', acc)
#Test accuracy with RNN: 0.82998

Defining and training an LSTM model, training embedding layer on the fly
Training the RNN
782/782 [==============================] - 181s 231ms/step - loss: 0.6933 - accuracy: 0.5000
Test accuracy with RNN: 0.5


### LSTM Model using pre-trained Embedding Layer

In [21]:
print("Defining and training an LSTM model, using pre-trained embedding layer")

rnnmodel2 = Sequential()
rnnmodel2.add(embedding_layer)
rnnmodel2.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
rnnmodel2.add(Dense(2, activation='sigmoid'))
rnnmodel2.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
print('Training the RNN')

rnnmodel2.fit(x_train, y_train,
          batch_size=32,
          epochs=1,
          validation_data=(x_val, y_val))
score, acc = rnnmodel2.evaluate(test_data, test_labels,
                            batch_size=32)
print('Test accuracy with RNN:', acc)
#Test accuracy with RNN: 0.793

Defining and training an LSTM model, using pre-trained embedding layer
Training the RNN
782/782 [==============================] - 133s 170ms/step - loss: 0.6931 - accuracy: 0.4999
Test accuracy with RNN: 0.49987998604774475


### Transformer Model 
Refer to the [keras tutorial](https://keras.io/examples/nlp/text_classification_with_transformer/) to implement and evaluate your model 

In [17]:
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow.keras as keras

In [18]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output)
        return self.layernorm2(out1 + ffn_output)

In [19]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super().__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [27]:
from tensorflow import keras
from tensorflow.keras import layers

num_heads = 4
ff_dim = 64

inputs = layers.Input(shape=(MAX_LEN,))
embedding_layer = TokenAndPositionEmbedding(MAX_LEN, MAX_NUM_WORDS, EMBEDDING_DIM)
x = embedding_layer(inputs)

for _ in range(2):
    transformer_block = TransformerBlock(EMBEDDING_DIM, num_heads, ff_dim)
    x = transformer_block(x)

x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(64, activation="relu")(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(2, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(
    loss="binary_crossentropy", 
    optimizer=keras.optimizers.optimizers.legacy.Adam(learning_rate=1e-4),
    metrics=["accuracy"]
)

model.fit(
    x_train, y_train, 
    batch_size=32, 
    epochs=2,
    validation_data=(x_val, y_val)
)

score, acc = model.evaluate(test_data, test_labels, batch_size=32)
print('Test accuracy with Transformer:', acc)
# Test accuracy with Transformer: 0.8668400049209595

Epoch 1/2
625/625 [==============================] - 1807s 3s/step - loss: 0.6999 - accuracy: 0.5092 - val_loss: 0.6928 - val_accuracy: 0.4906
Epoch 2/2
782/782 [==============================] - 815s 1s/step - loss: 0.3208 - accuracy: 0.8668
Test accuracy with Transformer: 0.8668400049209595
